Name: **Prasanna Sundararajan Muthukumaran**

UTA ID: **1001937587**

## **Image Classification With Deep Learning - Food101 Dataset**

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install lightning-bolts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Importing all the necessary packages

import torch
from torchvision.datasets import CIFAR10
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torchvision import transforms
import pytorch_lightning as pl
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.utils.data as data
from torchvision.datasets import Food101
import torchvision.models as models

/usr/local/lib/python3.8/dist-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised

In [4]:
class Food101Module(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.num_classes = 101
    
    def prepare_data(self):
        self.food101_train = Food101(self.data_dir, split= 'train', transform=transforms.Compose([transforms.RandAugment(), transforms.RandomCrop(56), transforms.ToTensor(), ]),download=True)
        self.food101_test = Food101(self.data_dir, split= 'test', transform=transforms.Compose([transforms.RandAugment(), transforms.RandomCrop(56), transforms.ToTensor(), ]), download=True)
        print('Length Of Train Dataset:', len(self.food101_train))
        print('Length Of Test Dataset:', len(self.food101_test))
    
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            food101_full = Food101(self.data_dir, split= 'train', transform=self.transform)
            food101_train_size = int(len(self.food101_train) * 0.8)
            food101_val_size = len(self.food101_train) - food101_train_size
            self.food101_train, self.food101_val = data.random_split(self.food101_train, [food101_train_size, food101_val_size])

            print('='*50)
            print('Train data set:', len(self.food101_train))
            print('Test data set:', len(self.food101_test))
            print('Validation data set:', len(self.food101_val))


        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.food_test = Food101(self.data_dir, split= 'test', transform=self.transform)
    
    def train_dataloader(self):
        return DataLoader(self.food101_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.food101_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.food101_test, batch_size=self.batch_size)

In [5]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Basic CNN
class Basic_CNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5), # nn.Conv2d(1, 6, 5) -> nn.Conv2d(3, 6, 5)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.estimator = nn.Sequential(
            nn.Linear(1936, 320), # nn.Linear(256, 120) -> nn.Linear(400, 120)
            nn.ReLU(),
            nn.Linear(320, 100),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("Basic CNN Training Loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("Validation Accuracy", prec[0])
        self.log("Validation Loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("Basic CNN Test Accuracy", prec[0])
        self.log("Basic CNN Test Loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [12]:
# All Convolution Neural Network
class All_Conv_NN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 5), # nn.Conv2d(1, 6, 5) -> nn.Conv2d(3, 6, 5)
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.estimator = nn.Sequential(
            nn.Conv2d(6, 100, 3),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.estimator(x)
        x = x.flatten(1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("All Convolution NN Training Loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("Validation Accuracy", prec[0])
        self.log("Validation Loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("All Convolution NN Test Accuracy", prec[0])
        self.log("All Convolution NN Test Loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# **Regularization On All Convolution NN**

While using dropout, each feature map is either fully dropped or not dropped at all. Since pooling usually operates separately on each feature map, it shouldn't make a difference if dropout is applied before or after pooling. There are 2 dropout layers here, one before pooling and one after pooling.


In [15]:
# Regularization On All Convolution NN

class Regularization(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 5), # nn.Conv2d(1, 6, 5) -> nn.Conv2d(3, 6, 5)
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(16, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.estimator = nn.Sequential(
            nn.Conv2d(6, 101, 3),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.estimator(x)
        x = x.flatten(1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("Regularization Training Loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("Validation Accuracy", prec[0])
        self.log("Validation Loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        prec = accuracy(y_hat, y)

        self.log("Regularization Test Accuracy", prec[0])
        self.log("Regularization Test Loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [8]:
# Model Creation

food101_dm = Food101Module(batch_size = 32)
food101_dm.prepare_data()
food101_dm.setup()

food101_model_1 = Basic_CNN()
food101_model_2 = All_Conv_NN()
food101_model_3 = Regularization()

early_stop_callback = EarlyStopping(monitor="Validation Loss", mode="min", patience=5)
checkpoint_callback = ModelCheckpoint(monitor="Validation Loss", mode="min")

Length Of Train Dataset: 75750
Length Of Test Dataset: 25250
Train data set: 60600
Test data set: 25250
Validation data set: 15150


In [9]:
# Fitting & Testing Using Basic_CNN Model

trainer1 = pl.Trainer(callbacks = [early_stop_callback, checkpoint_callback], max_epochs = 1)
trainer1.fit(model = food101_model_1, datamodule = food101_dm)
trainer1.test(dataloaders = food101_dm.test_dataloader())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


Length Of Train Dataset: 75750
Length Of Test Dataset: 25250


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type       | Params
-----------------------------------------
0 | features  | Sequential | 2.9 K 
1 | estimator | Sequential | 652 K 
-----------------------------------------
655 K     Trainable params
0         Non-trainable params
655 K     Total params
2.621     Total estimated model params size (MB)


Train data set: 60600
Test data set: 25250
Validation data set: 15150


Sanity Checking: 0it [00:00, ?it/s]

...

Length Of Train Dataset: 75750
Length Of Test Dataset: 25250


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 Basic CNN Test Accuracy    0.9900990128517151
   Basic CNN Test Loss       4.615166664123535
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'Basic CNN Test Accuracy': 0.9900990128517151,
  'Basic CNN Test Loss': 4.615166664123535}]

In [ ]:
# Fitting & Testing Using All_Conv_NN Model

trainer = pl.Trainer(callbacks = [checkpoint_callback], max_epochs = 1)
trainer.fit(model = food101_model_2, datamodule = food101_dm)
trainer.test(dataloaders = food101_dm.test_dataloader())

In [18]:
# Fitting & Testing Using Regularization Model

early_stop_callback = EarlyStopping(monitor="Regularization Training Loss", mode="min", patience=5)
trainer = pl.Trainer(callbacks = [early_stop_callback, checkpoint_callback], max_epochs = 1)
trainer.fit(model = food101_model_3, datamodule = food101_dm)
trainer.test(dataloaders = food101_dm.test_dataloader())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Length Of Train Dataset: 75750
Length Of Test Dataset: 25250


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type       | Params
-----------------------------------------
0 | features  | Sequential | 10.0 K
1 | estimator | Sequential | 5.6 K 
-----------------------------------------
15.6 K    Trainable params
0         Non-trainable params
15.6 K    Total params
0.062     Total estimated model params size (MB)


Train data set: 60600
Test data set: 25250
Validation data set: 15150


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

RuntimeError: ignored

## **References**

[1] https://github.com/ajdillhoff/CSE6363/blob/main/deep_learning/pl_demo/LeNetModel.py

[2] https://wandb.ai/wandb/wandb-lightning/reports/Image-Classification-Using-PyTorch-Lightning-and-Weights-Biases--VmlldzoyODk1NzY

[3] https://github.com/ajdillhoff/CSE6363/blob/main/deep_learning/pytorch_intro.ipynb

[4] https://github.com/ajdillhoff/CSE6363/blob/main/deep_learning/lenet5.ipynb